## AGENCY BANKING ANALYSIS

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime

In [ ]:
# Connecting to the SQLite database
db_path = 'E:/cbs.db'
conn = sqlite3.connect(db_path)

### Fetching and analyzing deposits at Agents

In [ ]:
# SQL query for agency banking deposits
query_deposits = """
SELECT 
    x.AC_BRANCH,
    x.AC_NO,
    acc.ACCOUNT_CLASS,
    x.DRCR_IND,
    x.EXCH_RATE,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.PERIOD_CODE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 x
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE
LEFT JOIN 
    STTM_CUST_ACCOUNT acc ON x.AC_NO = acc.CUST_AC_NO
WHERE 
    x.PRODUCT IN ('CDAU', 'DAGD', 'SECD') AND
    x.TRN_CODE IN ('705', 'D23', 'S09') AND
    x.DRCR_IND = 'C' AND
    x.CUST_GL = 'A' AND 
    acc.ACCOUNT_CLASS NOT IN ('AGBC', 'AGBA')
"""

In [ ]:
# Creating a dataframe for all deposit entries
df_deposits = pd.read_sql_query(query_deposits, conn)

In [ ]:
df_deposits.head(3)

,AC_BRANCH,AC_NO,ACCOUNT_CLASS,DRCR_IND,EXCH_RATE,FINANCIAL_CYCLE,LCY_AMOUNT,PERIOD_CODE,PRODUCT,PRODUCT_DESC,TRN_CODE,TRN_DSC,TRN_DT,TRN_REF_NO,VALUE_DT
0,31,031810000193,HOPES,C,0.0,FY2025,910000.0,JAN,CDAU,Agency - Cash Deposit Onus,705,AGENT CASH DEPOSIT,01/02/2025 00:00:00,022CDAU25002000J,01/02/2025 00:00:00
1,31,031000006914,HOPES,C,0.0,FY2025,100000.0,JAN,CDAU,Agency - Cash Deposit Onus,705,AGENT CASH DEPOSIT,01/02/2025 00:00:00,022CDAU250020008,01/02/2025 00:00:00
2,31,031000004078,JISA,C,0.0,FY2025,800000.0,JAN,CDAU,Agency - Cash Deposit Onus,705,AGENT CASH DEPOSIT,01/02/2025 00:00:00,024CDAU2500200DX,01/02/2025 00:00:00


In [ ]:
# Exporting to All deposit entries to Excel
output_path = './../data/AGENCY_BANKING_DEPOSITS_ENTRIES.xlsx'
df_deposits.to_excel(output_path, index=False)

output_path

In [ ]:
# Creating a dataframe for all deposit entries
df_agent_to_agent = pd.read_sql_query(agent_to_agent, conn)

### Fetching and analyzing withdrawals at Agents

In [ ]:
# SQL query for withdrawals
query_withdrawals = """
SELECT 
    x.AC_BRANCH,
    x.AC_NO,
    x.DRCR_IND,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.PERIOD_CODE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 x 
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE 
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE  
WHERE 
    x.PRODUCT IN ('CAAU', 'DAWM', 'SECW') AND
    x.TRN_CODE IN ('728', 'D26', 'S01') AND
    x.CUST_GL = 'A' AND
    x.DRCR_IND = 'D'
"""

In [ ]:
df_withdrawals = pd.read_sql_query(query_withdrawals, conn)

In [ ]:
df_withdrawals.head()

,AC_BRANCH,AC_NO,DRCR_IND,FINANCIAL_CYCLE,LCY_AMOUNT,PERIOD_CODE,PRODUCT,PRODUCT_DESC,TRN_CODE,TRN_DSC,TRN_DT,TRN_REF_NO,VALUE_DT
0,30,030000046987,D,FY2025,350000.0,JAN,SECW,SELCOM CASH WITHDRAWAL,S01,SELCOM CASH WITHDRAW,01/02/2025 00:00:00,001SECW250020267,01/02/2025 00:00:00
1,31,031000006903,D,FY2025,20000.0,JAN,DAWM,DIGITAL AGENT MOBILE WITHDRAW,D26,DIGITAL AGENT MOBILE WITHDRAW,01/02/2025 00:00:00,031DAWM250020501,01/02/2025 00:00:00
2,30,030000043322,D,FY2025,100000.0,JAN,DAWM,DIGITAL AGENT MOBILE WITHDRAW,D26,DIGITAL AGENT MOBILE WITHDRAW,01/02/2025 00:00:00,030DAWM250020003,01/02/2025 00:00:00
3,31,022000025942,D,FY2025,100000.0,JAN,CAAU,Agency Cash Withdrawal - Onus,728,AGENT CASH WITHDRAW,01/02/2025 00:00:00,022CAAU25002000F,01/02/2025 00:00:00
4,31,031810007647,D,FY2025,20000.0,JAN,CAAU,Agency Cash Withdrawal - Onus,728,AGENT CASH WITHDRAW,01/02/2025 00:00:00,022CAAU25002000I,01/02/2025 00:00:00


In [ ]:
# Exporting to All Withdrawal entries to Excel
output_path = './../data/AGENCY_BANKING_WITHDRAWALS.xlsx'
df_deposits.to_excel(output_path, index=False)

output_path

### Summarizing daily trends of deposits and withdrawals

In [ ]:
# Convert TRN_DT from string to datetime
df_deposits['TRN_DT'] = pd.to_datetime(df_deposits['TRN_DT'], format="%m/%d/%Y %H:%M:%S")
df_withdrawals['TRN_DT'] = pd.to_datetime(df_withdrawals['TRN_DT'], format="%m/%d/%Y %H:%M:%S")

In [ ]:
# Create a DATE column (only date part, drop time)
df_deposits['DATE'] = df_deposits['TRN_DT'].dt.date
df_withdrawals['DATE'] = df_withdrawals['TRN_DT'].dt.date

In [ ]:
# Group deposits by DATE
deposits_summary = df_deposits.groupby('DATE').agg(
    Total_Deposit_Amount=('LCY_AMOUNT', 'sum'),
    Deposit_Transactions=('LCY_AMOUNT', 'count')
).reset_index()

# Group withdrawals by DATE
withdrawals_summary = df_withdrawals.groupby('DATE').agg(
    Total_Withdrawal_Amount=('LCY_AMOUNT', 'sum'),
    Withdrawal_Transactions=('LCY_AMOUNT', 'count')
).reset_index()

In [ ]:
# Merge the two summaries on DATE
summary = pd.merge(deposits_summary, withdrawals_summary, on='DATE', how='outer')

In [ ]:
# Format amounts with commas (optional)
summary['Total_Deposit_Amount'] = summary['Total_Deposit_Amount'].apply(lambda x: "{:,.2f}".format(x))
summary['Total_Withdrawal_Amount'] = summary['Total_Withdrawal_Amount'].apply(lambda x: "{:,.2f}".format(x))

In [ ]:
summary.head()

,DATE,Total_Deposit_Amount,Deposit_Transactions,Total_Withdrawal_Amount,Withdrawal_Transactions
0,2025-01-02,"689,394,084.00",1172,"466,381,508.00",1979
1,2025-01-03,"618,589,836.00",916,"299,588,200.00",1355
2,2025-01-04,"413,437,691.00",711,"277,535,508.00",970
3,2025-01-06,"997,233,147.00",1712,"504,242,026.00",1637
4,2025-01-07,"701,707,927.00",1019,"325,552,700.00",821


In [ ]:
# Save to Excel, Daily summary
output_path = './../data/daily_summary_agency_banking.xlsx'
summary.to_excel(output_path, index=False)

output_path

### Agent Ranking for deposits

In [ ]:
# We take agent debits during customer deposits
agent_debits = """
SELECT 
    x.AC_BRANCH,
    x.AC_NO,
    acc.ACCOUNT_CLASS,
    x.DRCR_IND,
    x.EXCH_RATE,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.PERIOD_CODE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 x
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE
LEFT JOIN 
    STTM_CUST_ACCOUNT acc ON x.AC_NO = acc.CUST_AC_NO
WHERE 
    x.PRODUCT IN ('CDAU', 'DAGD', 'SECD') AND
    x.TRN_CODE IN ('705', 'D23', 'S09') AND
    x.DRCR_IND = 'D' AND
    x.CUST_GL = 'A'
"""

In [ ]:
df_agent_deposit_debits = pd.read_sql_query(agent_debits, conn)

In [ ]:
merged_df = pd.merge(df_agent_deposit_debits, df_deposits, on='TRN_REF_NO', how='inner')

In [ ]:
# 1️⃣ Find the last entry with PRODUCT_x == 'CDAU'
last_cdau_date = merged_df.loc[merged_df['PRODUCT_x'] == 'CDAU', 'TRN_DT_x'].max()

# 2️⃣ Filter dataset from that date onwards
filtered_df = merged_df[merged_df['TRN_DT_x'] >= last_cdau_date]

# 3️⃣ Group by AC_NO_x and AC_BRANCH_x, sum LCY_AMOUNT_x
summary = filtered_df.groupby(['AC_NO_x', 'AC_BRANCH_x']).agg(
    Total_LCY_Amount=('LCY_AMOUNT_x', 'sum')
).reset_index()

# 4️⃣ Sort by Total_LCY_Amount descending
summary = summary.sort_values(by='Total_LCY_Amount', ascending=False)

# Optional: Format amount with commas
summary['Total_LCY_Amount'] = summary['Total_LCY_Amount'].apply(lambda x: "{:,.2f}".format(x))

# 5️⃣ Load branch details (only required columns)
branches = pd.read_csv('./../data_tables/STTM_BRANCH.csv', usecols=['BRANCH_CODE', 'BRANCH_NAME'])

# 6️⃣ Merge summary with branch names
summary = summary.merge(
    branches,
    how='left',
    left_on='AC_BRANCH_x',
    right_on='BRANCH_CODE'
)

# 7️⃣ Drop unneeded columns
summary.drop(['AC_BRANCH_x', 'BRANCH_CODE'], axis=1, inplace=True)

# 8️⃣ Rename columns as requested
summary.rename(columns={
    'AC_NO_x': 'AGENT ACC',
    'BRANCH_NAME': 'BRANCH',
    'Total_LCY_Amount': 'AMOUNT'
}, inplace=True)

# ✅ Show result
summary.head()

,AGENT ACC,AMOUNT,BRANCH
0,023000098398,"3,055,031,014.00",MBEYA CITY BRANCH
1,017000016617,"1,759,436,000.00",RWAGASORE AGENCY
2,013000044234,"1,633,478,500.00",KATORO AGENCY
3,023000022192,"1,610,655,019.00",MBEYA CITY BRANCH
4,008000152944,"1,461,595,840.00",SAM NUJOMA BRANCH


In [ ]:
# Save dataset to Excel
summary.to_excel('../data/agent_summary_with_branch_names.xlsx', index=False)

### Deposits: Analyzing Agent-to-Agent transfers Vs Customer-to-Agent

In [ ]:
# Querying all agent deposits(both agent-to-agent transfers and customer-to-agent)
all_deposits = """
SELECT 
    x.AC_BRANCH,
    x.AC_NO,
    acc.ACCOUNT_CLASS,
    x.DRCR_IND,
    x.EXCH_RATE,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.PERIOD_CODE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 x
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE
LEFT JOIN 
    STTM_CUST_ACCOUNT acc ON x.AC_NO = acc.CUST_AC_NO
WHERE 
    x.PRODUCT IN ('CDAU', 'DAGD', 'SECD') AND
    x.TRN_CODE IN ('705', 'D23', 'S09') AND
    x.DRCR_IND = 'C' AND
    x.CUST_GL = 'A'
"""

In [ ]:
# Creating a dataframe for all deposit entries
df_deposits_all = pd.read_sql_query(all_deposits, conn)

In [ ]:
# Convert TRN_DT_x to datetime
df_deposits_all['TRN_DT'] = pd.to_datetime(df_deposits_all['TRN_DT'], format="%d/%m/%Y %H:%M:%S", errors='coerce')

# Create DATE column (only date part)
df_deposits_all['DATE'] = df_deposits_all['TRN_DT'].dt.date

In [ ]:
# Classify transfers
def classify_transfer(row):
    if row['ACCOUNT_CLASS'] in ['AGBA', 'AGBC']:
        return 'Agent to Agent'
    else:
        return 'Customer to Agent'

df_deposits_all['TRANSFER_TYPE'] = df_deposits_all.apply(classify_transfer, axis=1)

In [ ]:
# Group by DATE and TRANSFER_TYPE
summary = df_deposits_all.groupby(['DATE', 'TRANSFER_TYPE']).agg(
    Total_LCY_Amount=('LCY_AMOUNT', 'sum'),
    Transaction_Count=('LCY_AMOUNT', 'count')
).reset_index()

In [ ]:
# Pivot so Agent to Agent and Customer to Agent appear side by side
pivot_summary = summary.pivot(index='DATE', columns='TRANSFER_TYPE', values=['Total_LCY_Amount', 'Transaction_Count']).fillna(0)

In [ ]:
# Flatten multi-level column names
pivot_summary.columns = ['_'.join(col).strip() for col in pivot_summary.columns.values]

In [ ]:
# Optional: Format amounts with commas
pivot_summary['Total_LCY_Amount_Agent to Agent'] = pivot_summary['Total_LCY_Amount_Agent to Agent'].apply(lambda x: "{:,.2f}".format(x))
pivot_summary['Total_LCY_Amount_Customer to Agent'] = pivot_summary['Total_LCY_Amount_Customer to Agent'].apply(lambda x: "{:,.2f}".format(x))

# Reset index to bring DATE back as a column
pivot_summary.reset_index(inplace=True)

In [ ]:
pivot_summary.head()

,DATE,Total_LCY_Amount_Agent to Agent,Total_LCY_Amount_Customer to Agent,Transaction_Count_Agent to Agent,Transaction_Count_Customer to Agent
0,2025-01-02,"434,384,238.00","481,761,370.00",354.0,860.0
1,2025-01-03,"363,955,059.00","568,004,346.00",232.0,889.0
2,2025-02-01,"248,621,175.00","689,394,084.00",244.0,1172.0
3,2025-02-04,"1,352,506,045.00","1,322,517,506.00",814.0,2681.0
4,2025-02-05,"1,295,273,787.00","1,301,500,328.00",1124.0,2562.0


In [ ]:
# Save dataset to Excel
pivot_summary.to_excel('../data/agent_to_agent_trend.xlsx', index=False)

In [ ]:
# Close DB connection
conn.close()